# Please ignore Question 1 part and start reviewing Question 2 below

# Question 1

For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

1. Start by creating a new Notebook for this assignment.
2. Use the Notebook to build the code to scrape the following Wikipedia page, 

https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, 

in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

3. To create the above dataframe:

a. The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

b. Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

c. More than one neighborhood can exist in one postal code area. 
For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. 
These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

d. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 
So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

e. Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.

e. In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

4. Submit a link to your Notebook on your Github repository. (10 marks)

Note: There are different website scraping libraries and packages in Python. For scraping the above table, you can simply use pandas to read the table into a pandas dataframe.

Another way, which would help to learn for more complicated cases of web scraping is using the BeautifulSoup package. Here is the package's main documentation page: http://beautiful-soup-4.readthedocs.io/en/latest/

The package is so popular that there is a plethora of tutorials and examples on how to use it. Here is a very good Youtube video on how to use the BeautifulSoup package: https://www.youtube.com/watch?v=ng2o98k983k

Use pandas, or the BeautifulSoup package, or any other way you are comfortable with to transform the data in the table on the Wikipedia page into the above pandas dataframe.

__read the Wikipedia page of postal codes of Canada using pandas__

In [7]:
import pandas as pd

#header=0 tells pandas to use the first row as column names
tables = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M", header=0)

#tables[0] contain our postal codes data
print("The shape is : ", tables[0].shape)
tables[0].head()

The shape is :  (288, 3)


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Only process the cells that have an assigned borough. 

Ignore cells with a borough that is Not assigned.

In [8]:
toronto_postal_info = tables[0][tables[0]["Borough"] != "Not assigned"]

#rename Postcode and Neighbourhood columns to PostalCode and Neighborhood
toronto_postal_info.rename(columns={"Postcode":"PostalCode", "Neighbourhood":"Neighborhood"}, inplace=True)
toronto_postal_info.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


More than one neighborhood can exist in one postal code area. 

For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. 

These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [9]:
#combine mutiple neighborhoods separated by a comma for the same PostalCode
postal_code_neigh_series = toronto_postal_info.groupby(['PostalCode'])['Neighborhood'].apply(lambda neigh: ','.join(neigh))

#convert above series to pandas df
postal_code_neigh_df = pd.DataFrame(postal_code_neigh_series).reset_index()
postal_code_neigh_df.head()

,PostalCode,Neighborhood
0,M1B,"Rouge,Malvern"
1,M1C,"Highland Creek,Rouge Hill,Port Union"
2,M1E,"Guildwood,Morningside,West Hill"
3,M1G,Woburn
4,M1H,Cedarbrae


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 

So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [10]:
#first check how many rows have Not assigned neighborhood
neigh_not_assignd = toronto_postal_info[toronto_postal_info["Neighborhood"] == "Not assigned"]
neigh_not_assignd.shape

(1, 3)

In [11]:
#display neigh_not_assignd
neigh_not_assignd

,PostalCode,Borough,Neighborhood
8,M7A,Queen's Park,Not assigned


In [12]:
#now make neighborhood the same as the borough for rows that have Not assigned neighborhood (only 1 row is such, but below code is for a general case of multiple rows)
import numpy as np

print("Before change: ", toronto_postal_info.loc[8])

toronto_postal_info['Neighborhood'] = np.where(toronto_postal_info.Neighborhood == "Not assigned",toronto_postal_info.Borough,toronto_postal_info.Neighborhood)

print("After change: ", toronto_postal_info.loc[8])

Before change:  PostalCode               M7A
Borough         Queen's Park
Neighborhood    Not assigned
Name: 8, dtype: object
After change:  PostalCode               M7A
Borough         Queen's Park
Neighborhood    Queen's Park
Name: 8, dtype: object


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


now append postal_code_neigh_df to toronto_postal_info so we can have combined mutiple neighborhoods separated by a comma for the same PostalCode

In [13]:
toronto_postal_info_init = pd.merge(toronto_postal_info, postal_code_neigh_df, on='PostalCode', suffixes=('_1', '_2'))

#we need to replace Not assigned value in Neighborhood_2 column with respective value in Neighborhood_1 column
print("Before change: ", toronto_postal_info_init.loc[6])

toronto_postal_info_init['Neighborhood_2'] = np.where(toronto_postal_info_init.Neighborhood_2 == "Not assigned",toronto_postal_info_init.Neighborhood_1,toronto_postal_info_init.Neighborhood_2)

print("After change: ", toronto_postal_info_init.loc[6])

toronto_postal_info_init.head(20)

Before change:  PostalCode                 M7A
Borough           Queen's Park
Neighborhood_1    Queen's Park
Neighborhood_2    Not assigned
Name: 6, dtype: object
After change:  PostalCode                 M7A
Borough           Queen's Park
Neighborhood_1    Queen's Park
Neighborhood_2    Queen's Park
Name: 6, dtype: object


,PostalCode,Borough,Neighborhood_1,Neighborhood_2
0,M3A,North York,Parkwoods,Parkwoods
1,M4A,North York,Victoria Village,Victoria Village
2,M5A,Downtown Toronto,Harbourfront,"Harbourfront,Regent Park"
3,M5A,Downtown Toronto,Regent Park,"Harbourfront,Regent Park"
4,M6A,North York,Lawrence Heights,"Lawrence Heights,Lawrence Manor"
5,M6A,North York,Lawrence Manor,"Lawrence Heights,Lawrence Manor"
6,M7A,Queen's Park,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue,Islington Avenue
8,M1B,Scarborough,Rouge,"Rouge,Malvern"
9,M1B,Scarborough,Malvern,"Rouge,Malvern"


__Final dataframe__

In [14]:
#drop Neighborhood_1 column since we will use Neighborhood_2 column
toronto_postal_info_init.drop(columns=['Neighborhood_1'], axis=1, inplace=True)
#rename Neighborhood_2 column to Neighborhood
toronto_postal_info_init.rename(columns={'Neighborhood_2':'Neighborhood'}, inplace=True)
#keep unique rows and reset the index
toronto_postal_info_final= toronto_postal_info_init.drop_duplicates().reset_index(drop=True)
toronto_postal_info_final.head(20)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe

In [15]:
print("The number of rows is: ",toronto_postal_info_final.shape[0])

The number of rows is:  103


# Question 2

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code. 

Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

Use the Geocoder package or the csv file to create the dataframe.

__Read Geospatial_Coordinates.csv into a pandas dataframe__

In [1]:
# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials = {
    'IAM_SERVICE_ID': 'iam-ServiceId-b864c9e0-faf2-4a63-bc11-5d1f0358065a',
    'IBM_API_KEY_ID': 'TkYgQ4R_XrCSKn2qHharz1E9wVFTSlDvr69ZQRJxf4Kl',
    'ENDPOINT': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT': 'https://iam.ng.bluemix.net/oidc/token',
    'BUCKET': 'myproject-donotdelete-pr-sndcpzislvwjid',
    'FILE': 'Geospatial_Coordinates.csv'
}

In [2]:
from ibm_botocore.client import Config
import ibm_boto3
cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])

In [3]:
from ibm_botocore.client import Config
import ibm_boto3
def download_file_cos(credentials,local_file_name,key):  
    cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])
    try:
        res=cos.download_file(Bucket=credentials['BUCKET'],Key=key,Filename=local_file_name)
    except Exception as e:
        print(Exception, e)
    else:
        print('File Downloaded')

In [5]:
cos.download_file(Bucket=credentials['BUCKET'],Key='Geospatial_Coordinates.csv',Filename='Geospatial_Coordinates1.csv')

import pandas as pd 
# Read data from file 'filename.csv' 
# (in the same directory that your python process is based)
# Control delimiters, rows, column names with read_csv (see later) 
geo_spatial_data = pd.read_csv("Geospatial_Coordinates1.csv") 
# Preview the first 5 lines of the loaded data 
geo_spatial_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


geo_spatial_data df has the same number of rows as toronto_postal_info_final

In [16]:
print("The number of rows in geo_spatial_data is ", geo_spatial_data.shape[0])
print("The number of rows toronto_postal_info_final is ",toronto_postal_info_final.shape[0])

The number of rows in geo_spatial_data is  103
The number of rows toronto_postal_info_final is  103


__Append Latitude and	Longitude to toronto_postal_info_final and create toronto_postal_info_final2 df__

In [22]:
#first in geo_spatial_data df we rename Postal Code to PostalCode 
geo_spatial_data.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
    
#then we merge toronto_postal_info_final and geo_spatial_data on PostalCode
toronto_postal_info_final2 = pd.merge(toronto_postal_info_final, geo_spatial_data, on='PostalCode')
toronto_postal_info_final2.head(20)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937


In [27]:
#there are no missing values in the toronto_postal_info_final2
toronto_postal_info_final2.isnull().sum()

PostalCode      0
Borough         0
Neighborhood    0
Latitude        0
Longitude       0
dtype: int64

In [28]:
#the number of rows is also preserved
print("The number of rows toronto_postal_info_final2 is ",toronto_postal_info_final2.shape[0])

The number of rows toronto_postal_info_final2 is  103
